In [32]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import time
import random
import re
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from unidecode import unidecode
import xml.etree.ElementTree as ET
from cachecontrol import CacheControl
from requests.cookies import RequestsCookieJar
from googletrans import Translator
from medscraper.tools import tipo_cambio
from medscraper.tools import navegador
from medscraper.tools import tidy
import pkg_resources

In [33]:
options=webdriver.ChromeOptions()
options.add_argument('--incognito')
options.add_argument('--disable-blink-features=AutomationControlled')
options.add_argument('--disable-cache')
options.add_argument('--disable-cookies')
options.add_argument('--headless')
        
#Desabilitar cache
no_cookies = RequestsCookieJar()
session=CacheControl(requests.session())
session.cookies=no_cookies

In [34]:
#Meds a buscar
meds=pd.read_excel("piloto_seguimiento/busqueda_canada.xlsx")
meds

,med,principio,strength
0,acalabrutinib,acalabrutinib,100+mg
1,afatinib,afatinib,20+mg
2,afatinib,afatinib,30+mg
3,afatinib,afatinib,40+mg
4,ambrisentan,ambrisentan,5+mg
5,ambrisentan,ambrisentan,10+mg
6,docetaxel,docetaxel,20+mg
7,docetaxel,docetaxel,80+mg%252f4+ml
8,everolimus,everolimus,0.25+mg
9,everolimus,everolimus,0.5+mg


In [35]:
muestra=meds.sample(3)
muestra

,med,principio,strength
17,venclyxto,venetoclax,10+mg
3,afatinib,afatinib,40+mg
21,capecitabine,capecitabina,500+mg


In [36]:
url_basica="https://www.pharmacychecker.com/"
fin="/#prices"
driver=webdriver.Chrome(service=Service("C:/Users/claudio.pacheco/chromedriver.exe"), options=options)

In [54]:
def precios_pharmacychecker():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    medida=soup.find("div", {"id": "drug-strength"}).text
    results=soup.find_all("li", {"class": "drug-pricing-item mt-2"})
    return results, medida

def data_pcheck(results,medida):
    df = pd.DataFrame(columns=['precio', 'farmacia', 'producto', 'quantity', 'medida', 'ship'])
    for result in results:
        price = result.find('p', {'class': 'desktop-price'}).text
        pharmacy = result.find('a', {'class': 'cpc-drug-listings-link-image'}).get("data-pharm")
        producto = result.find('a', {'class': 'cpc-drug-listings-link-image'}).get("data-drug")
        cantidad = result.find('div', {'class': 'col-3 col-lg-3 drug-pricing-details hide-on-mobile'}).text
        ship = result.find('span', {'class': 'drug-pricing-shipping-info'}).text
        df = pd.concat([df, pd.DataFrame({'producto': producto,'precio': price, 'farmacia': pharmacy, 
                                           'quantity': cantidad, 
                                          'medida': medida, 'ship': ship}, index=[0])], ignore_index=True)
    return df

#Diccionario de farmacias
canada_pharmacies = ['liferxpharmacy', 'canadianprescriptiondrugstore', 'medsengage', 
                     'offshorecheapmeds', 'pricepropharmacy', 'canadianpharmacystore', 
                     'discountcanadadrugs', 'spfpharmacy', '1800rxonline.com', 'buylowdrugs']

In [58]:
master_df=pd.DataFrame()
for index, row in muestra.iterrows():
    med = row["med"]
    strength = row["strength"]
    principio_value = row["principio"]
    driver.get(url_basica + med + "/" + strength + fin)
    html=driver.page_source
    soup=BeautifulSoup(html,'html.parser')
    resultados,medida=precios_pharmacychecker()
    df = data_pcheck(resultados, medida)
    df["principio"]=principio_value

    # Pegar la info de la muestra
    master_df = pd.concat([master_df, df], ignore_index=True)
    #Limpiar
    #pais
    master_df["pais"]=master_df["farmacia"].apply(lambda x: "Canada" if any(pharmacy in x for pharmacy in canada_pharmacies) else "Estados Unidos")
    master_df['producto'] = master_df['producto'].apply(tidy.clean_text)
                #Traducir del inglés al español
    translator = Translator()
    master_df['producto'] = master_df['producto'].apply(lambda x: translator.translate(x, src='en', dest='es').text)
    master_df['producto'] = master_df['producto'].apply(tidy.clean_text)

    master_df["precio"]=master_df["precio"].apply(tidy.clean_text)
    master_df['ship'] = master_df['ship'].astype(str)
    master_df["ship"]=master_df["ship"].str.replace("\n","")
    master_df["ship"]=master_df["ship"].str.split("Shipping").str[0]
    #Extraer números con decimales
    master_df['ship'] = master_df['ship'].str.findall(r'(\d+\.\d+)').apply(lambda x: x[0] if len(x)>0 else None)
    #Sustituir None por 0
    master_df['ship'] = master_df['ship'].fillna(0)
    #Convertir a float
    master_df['ship'] = master_df['ship'].astype(float)
    
    


In [59]:
master_df

,precio,farmacia,producto,quantity,medida,ship,principio,pais
0,127.80,1800rxonline.com/,venclyxto,14 tablets,10 mg,0.00,venetoclax,Canada
1,118.00,affordablerxmeds.com,venclyxto,14 tablets,10 mg,18.00,venetoclax,Estados Unidos
2,2842.72,liferxpharmacy.com/,afatinib,56 tablets,40 mg,9.00,afatinib,Canada
3,3873.17,liferxpharmacy.com/,afatinib,84 tablets,40 mg,9.00,afatinib,Canada
4,3343.74,offshorecheapmeds.com,afatinib,56 tablets,40 mg,9.95,afatinib,Canada
5,4556.03,offshorecheapmeds.com,afatinib,84 tablets,40 mg,9.95,afatinib,Canada
6,96.51,1800rxonline.com/,capecitabina,30 tablets,500 mg,0.00,capecitabina,Canada
7,188.00,affordablerxmeds.com,capecitabina,QTY: 120,500 mg,18.00,capecitabina,Estados Unidos
8,390.99,affordablerxmeds.com,capecitabina,QTY: 360,500 mg,18.00,capecitabina,Estados Unidos
9,148.00,buylowdrugs.com/,capecitabina,120 tablets,500 mg,0.00,capecitabina,Canada
